# Capstone Team 8 Face Identification & Emotion Recognition

## Module Installation

In [ ]:
!pip install deepface

In [ ]:
!pip install google-colab

In [ ]:
!pip install flask flask-cors

In [ ]:
!pip install pyngrok

In [ ]:
!ngrok authtoken 2j2W6kUSi8ffERk2o7l38R5HTes_7FckvEsyLoehx143yX2W3

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from deepface import DeepFace
from matplotlib import pyplot as plt
import cv2
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
import os
from google.colab import files

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from deepface import DeepFace
import os
import shutil
from pyngrok import ngrok

app = Flask(__name__)
CORS(app)

source_photo_directory = "drive/MyDrive/source_photos"
login_photo_directory = "drive/MyDrive/login_photos"

# Create directories if they don't exist
os.makedirs(source_photo_directory, exist_ok=True)
os.makedirs(login_photo_directory, exist_ok=True)

@app.route('/upload', methods=['POST'])
def upload_photo():
    user_type = request.form.get('user_type')
    user_name = request.form.get('user_name')
    photo = request.files.get('photo')

    if not user_type or not user_name or not photo:
        return jsonify({"error": "Missing required parameters"}), 400

    if user_type not in ['signup', 'login']:
        return jsonify({"error": "Invalid user type"}), 400

    # Save the uploaded photo
    photo_path = os.path.join('/content', photo.filename)
    photo.save(photo_path)

    if user_type == 'signup':
        # Save the photo to the source directory with the username
        new_photo_path = os.path.join(source_photo_directory, user_name.replace(" ", "_") + os.path.splitext(photo.filename)[1])
        shutil.move(photo_path, new_photo_path)
        return jsonify({"message": f"Welcome {user_name}, your photo has been saved."})

    elif user_type == 'login':
        # Save the photo to the login directory
        new_photo_path = os.path.join(login_photo_directory, photo.filename)
        shutil.move(photo_path, new_photo_path)

        try:
            # Perform face recognition
            matched_photo = find_identity(new_photo_path, source_photo_directory)
            if matched_photo:
                username = extract_username_from_path(matched_photo)
                emotion = analyze_emotion(new_photo_path)
                return jsonify({"message": f"Login Successful, welcome back {username}! You seem to be feeling {emotion}."})
            else:
                return jsonify({"message": "No matching user found. Please ensure your photo is clear or sign up if you haven't yet."})
        except Exception as e:
            return jsonify({"error": str(e)}), 500

def find_identity(photo_path, database_path):
    """
    Finds the identity of a person in a photo using facial recognition.

    Parameters:
        photo_path (str): The file path of the photo containing the person's face.
        database_path (str): The file path of the database containing reference face embeddings.

    Returns:
        str or None: The identity of the person if found in the database, or None if not found.
    """
    try:
        # Switch face identification models by changing model_name
        df = DeepFace.find(img_path=photo_path, db_path=database_path, model_name='VGG-Face')
        if len(df) > 0:
            identity = df[0].loc[0,'identity']
            return identity
        else:
            return None
    except Exception as e:
        print(f"Error during facial recognition: {e}")
        return None

def extract_username_from_path(path):
    """
    Extract the username from a file path.

    Parameters:
        path (str): The file path from which to extract the username.

    Returns:
        str: The extracted username.
    """
    filename = os.path.basename(path)
    username = os.path.splitext(filename)[0].replace("_", " ")
    return username

def analyze_emotion(photo_path):
    """
    Analyze the dominant emotion in a photo.

    Parameters:
        photo_path (str): The file path of the photo to be analyzed.

    Returns:
        str: The dominant emotion detected in the photo.
    """
    predictions = DeepFace.analyze(img_path=photo_path)
    emotion = predictions[0]['dominant_emotion']
    return emotion

if __name__ == '__main__':
    # Authenticate ngrok
    ngrok.set_auth_token("2j2W6kUSi8ffERk2o7l38R5HTes_7FckvEsyLoehx143yX2W3")

    # Start ngrok tunnel
    ngrok_tunnel = ngrok.connect(5000)
    ngrok_url = ngrok_tunnel.public_url
    print(f"Public URL: {ngrok_url}")

    # Start Flask app
    app.run(host='0.0.0.0', port=5000)
